In [1]:
import oracledb as od
import pandas as pd
import numpy as np
import config
from tqdm import tqdm
import ast
import re
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from scipy.linalg import svd

In [2]:
def load_recipe(n =1000):
    od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12") # db connection
    conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],  dsn = config.DB_CONFIG['dsn'])
    exe = conn.cursor()
    exe.execute(f'select * from (select * from recipe_table order by row_cnt asc) where row_cnt <= {n}')
    row = exe.fetchall() # row 불러오기
    column_name = exe.description # column 불러오기
    columns=[]
    for i in column_name:
        columns.append(i[0])
    result = pd.DataFrame(row, columns=columns) # row, column을 pandas DataFrame으로 나타내기
    result.rename(mapper=str.lower, axis='columns', inplace=True)
    conn.close()
    return result

In [3]:
def select_table(query):
    od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12") # db connection
    conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],  dsn = config.DB_CONFIG['dsn'])
    exe = conn.cursor()
    exe.execute(query)
    row = exe.fetchall() # row 불러오기
    column_name = exe.description # column 불러오기
    columns=[]
    for i in column_name:
        columns.append(i[0])
    result = pd.DataFrame(row, columns=columns) # row, column을 pandas DataFrame으로 나타내기
    result.rename(mapper=str.lower, axis='columns', inplace=True)
    conn.close()
    return result

In [4]:
query = 'select * from nutrient_table'
nutri_df = select_table(query)
nutri_df

,대표식품명,에너지(kcal),수분(g),단백질(g),지방(g),회분(g),탄수화물(g),당류(g),식이섬유(g),칼슘(mg),...,베타카로틴(μg),티아민(mg),리보플라빈(mg),니아신(mg),비타민c(mg),비타민d(μg),콜레스테롤(mg),포화지방산(g),트랜스지방산(g),폐기율(%)
0,보리밥,150,62.1,3.93,0.75,0.28,32.97,0.53,3.4,14.0,...,0.0,0.09,0.013,0.26,0.0,0.0,0.0,0.18,0.0,0.0
1,수수,376,9.4,12.73,3.02,1.65,73.19,0.66,11.6,8.0,...,4.0,0.62,0.016,1.59,0.2,0.0,0.0,0.43,0.0,0.0
2,전분,356,13.9,0.20,0.20,0.10,85.60,0.00,0.0,18.0,...,NaN,0.00,0.000,0.00,0.0,0.0,0.0,NaN,NaN,0.0
3,천마,81,78.7,1.10,0.00,0.90,19.30,NaN,NaN,16.0,...,5.0,0.11,0.040,0.50,96.0,NaN,NaN,NaN,NaN,15.0
4,칡뿌리,137,63.7,2.48,0.10,1.67,32.05,3.58,4.4,383.0,...,18.0,0.15,0.020,1.32,4.7,0.0,0.0,0.04,0.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,목이버섯,18,91.2,1.02,0.43,0.39,7.01,0.10,3.4,4.0,...,0.0,0.00,0.000,0.06,0.0,0.0,0.0,0.06,0.0,0.0
696,밤버섯,17,91.7,2.90,0.90,1.00,3.50,NaN,NaN,41.0,...,0.0,0.40,0.060,38.40,2.0,NaN,NaN,NaN,NaN,0.0
697,버들송이버섯,183,7.6,14.30,2.70,4.30,71.10,NaN,NaN,17.0,...,0.0,0.36,1.250,16.90,0.0,NaN,NaN,NaN,NaN,0.0
698,뽕나무버섯,182,4.4,12.80,0.60,5.60,76.60,NaN,NaN,12.0,...,0.0,0.26,1.130,7.00,0.0,NaN,NaN,NaN,NaN,0.0


In [5]:
raw_data = load_recipe(n = 1000)

In [6]:
raw_data

,recipe_url,recipe_title,recipe_summary1,recipe_summary2,recipe_summary3,recipe_ingredients,recipe_step,recipe_tag,recipe_comment,recipe_view,recipe_category_type,recipe_category_method,recipe_category_situation,row_cnt
0,http://www.10000recipe.com/recipe/6853904,호박 새우젓찌개 만드는 방법~★,3인분,30분 이내,초급,"{'재료': ['둥근호박1개', '청양고추3개', '당근약간', '실파약간', '새...","['#1 먼저 둥근호박 하나를 반달썰기로 썰어 냄비에 넣고', '#2 새우젓 1/2...","['새우젓호박찌개', '새우젓호박째개만드는방법', '새우젓찌개', '호박찌개', '...",1.0,6517.0,찌개,끓이기,일상,1
1,http://www.10000recipe.com/recipe/6853845,바글바글 전복 된장찌개,4인분,30분 이내,아무나,"{'재료': ['된장1스푼', '전복'], '부재료와 양념': ['양파1/2개', ...","['#1 된장 한 큰 술 넣어주었어요.', '#2 전복 듬뿍, 양파 1/2개, 둥근...","['된짱찌개', '된짱찌개맛있게끓이는법', '된장찌개레시피', '된장찌개끓이는법',...",0.0,7019.0,찌개,끓이기,일상,2
2,http://www.10000recipe.com/recipe/6853803,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,None,None,None,"{'재료': ['순두부(연두부)', '바지락30g', '미더덕30g', '멸치육수(...",[],"['순두부찌개', '순두부', '미더덕', '바지락', '달걀', '고추기름', '...",0.0,3054.0,찌개,끓이기,일상,3
3,http://www.10000recipe.com/recipe/6849181,정통 바지락 순두부 찌개,2인분,60분 이내,중급,"{'주재료': ['순두부1봉', '바지락100g', '양파50g', '애호박10g'...","['#1 뚝배기에 양파,다진 마늘,바지락을 넣어 중불에서 살짝 볶는다.', '#2 ...",[],0.0,4217.0,찌개,끓이기,일상,4
4,http://www.10000recipe.com/recipe/6848906,크림토마토 비프 스튜,2인분,60분 이내,중급,"{'주재료': ['쇠고기등심250g', '파프리카1개', '양파1/3개', '마늘2...","['#1 쇠고기는 2cm 크기로 네모나게 썰어 소금과 후추로 밑간한다.', '#2 ...",[],0.0,2053.0,찌개,끓이기,일상,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,http://www.10000recipe.com/recipe/1583341,버섯 와인 닭조림,4인분,60분 이내,초급,"{'재료': ['토종닭1마리', '백일송이1봉지', '베이컨10장', '그린빈스',...","['#1 적당한 크기로 자른 토종닭은 흐르는 물에 잘 헹궈낸 후 소금, 후추를 조금...",[],3.0,4017.0,메인반찬,조림,일상,996
996,http://www.10000recipe.com/recipe/6876217,복숭아 우유 아이스크림~*,1인분,10분 이내,아무나,"{'재료': ['우유1종이컵', '복숭아1개', '연유취향껏']}","['#1 우유 얼음 틀에 붓고 얼려 주세요', '#2 복숭아 껍질을 깍아내고듬성 듬...","['아이스크림만들기', '디저트', '복숭아', '간식', '우유아이스크림', '샤...",2.0,6394.0,디저트,기타,일상,997
997,http://www.10000recipe.com/recipe/3950629,토마토 아이스바,2인분,10분 이내,아무나,"{'재료': ['토마토2개', '꿀2스푼', '소금1꼬집', '레몬즙(생략가능)1작...","['#1 잘 익은 토마토를 꼭지를 제거한 후, 잘 씻어서 준비합니다', '#2 토마...",[],1.0,7213.0,디저트,기타,일상,998
998,http://www.10000recipe.com/recipe/6928679,< 간식> 감자칩 / 전자렌지,2인분,15분 이내,초급,"{'재료': ['감자3개'], '양념': ['케첩적당량']}","['#1 감자를 줄무늬칼로 잘라 주세요', '#2 전자렌지에 2분 돌려주세요전자레인...","['감자칩 만들기', '전자렌지', '간식']",0.0,1638.0,디저트,기타,일상,999


In [7]:
def recipe_preprocessing(raw):
    data = raw.loc[raw['recipe_ingredients'].notnull()].copy()  # None 값 제거
    def clean_ingredients(ingredients):
        if ingredients is not None:
            ingredients = ingredients.replace('\\ufeff', '').replace('\\u200b', '')
        return ingredients

    def not_empty_ingredients(row):
        return row['recipe_ingredients'].strip() != '{}' # 결측치 제거

    data["recipe_ingredients"] = data["recipe_ingredients"].apply(clean_ingredients)
    data = data[data.apply(not_empty_ingredients, axis=1)]
    result = data[['recipe_title', 'recipe_ingredients']]

    title_idx = result[result['recipe_title'].isnull()].index
    del_idx = result[result['recipe_ingredients'].str.startswith('소시지')].index
    result.drop(del_idx, inplace=True)
    result.drop(title_idx, inplace=True)

    return result


In [8]:
recipe = recipe_preprocessing(raw_data)
recipe

C:\Users\admin\AppData\Local\Temp\ipykernel_2312\3225395148.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(del_idx, inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_2312\3225395148.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(title_idx, inplace=True)


,recipe_title,recipe_ingredients
0,호박 새우젓찌개 만드는 방법~★,"{'재료': ['둥근호박1개', '청양고추3개', '당근약간', '실파약간', '새..."
1,바글바글 전복 된장찌개,"{'재료': ['된장1스푼', '전복'], '부재료와 양념': ['양파1/2개', ..."
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,"{'재료': ['순두부(연두부)', '바지락30g', '미더덕30g', '멸치육수(..."
3,정통 바지락 순두부 찌개,"{'주재료': ['순두부1봉', '바지락100g', '양파50g', '애호박10g'..."
4,크림토마토 비프 스튜,"{'주재료': ['쇠고기등심250g', '파프리카1개', '양파1/3개', '마늘2..."
...,...,...
995,버섯 와인 닭조림,"{'재료': ['토종닭1마리', '백일송이1봉지', '베이컨10장', '그린빈스',..."
996,복숭아 우유 아이스크림~*,"{'재료': ['우유1종이컵', '복숭아1개', '연유취향껏']}"
997,토마토 아이스바,"{'재료': ['토마토2개', '꿀2스푼', '소금1꼬집', '레몬즙(생략가능)1작..."
998,< 간식> 감자칩 / 전자렌지,"{'재료': ['감자3개'], '양념': ['케첩적당량']}"


In [9]:
#1. 식재료 단위 별로 쪼개기
def split_ingredient(data):
    for i in range(1, 21):
        data.loc[:, f'ingredient{i}'] = None
        data.loc[:, f'quantity{i}'] = None
        data.loc[:, f'unit{i}'] = None
   
    non_matching_items = {} # 패턴과 일치하지 않는 데이터를 저장할 딕셔너리

    for idx, row in tqdm(data.iterrows(), total=data.shape[0]): #tqdm으로 진행상황 확인
        if row['recipe_ingredients']:
            ingredients_dict = ast.literal_eval(row["recipe_ingredients"]) #딕셔너리 형태로 저장된 recipe_ingredients 불러오기
            ingredient_count = 1
            for category, items in ingredients_dict.items(): #category : 재료, 양념재료, items: 사과1개, 돼지고기600g
                if items:  # 아이템이 존재하는 경우
                    for item in items:
                        match = re.match(r'([가-힣a-zA-Z]+(\([가-힣]+\))?)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item) # 정규식
                        if match:
                            ingredient, _, quantity, unit = match.groups()
                            
                            data.loc[idx, f'ingredient{ingredient_count}'] = ingredient
                            data.loc[idx, f'quantity{ingredient_count}'] = quantity
                            data.loc[idx, f'unit{ingredient_count}'] = unit

                            ingredient_count += 1
                        else:
                            # 패턴과 일치하지 않는 경우 딕셔너리에 추가
                            non_matching_items[idx] = item
        else:
            pass

    # 패턴과 일치하지 않는 데이터 출력 X => 날려버리기!
    for idx, item in non_matching_items.items():
        print(f'Row {idx}: {item}')
    return data

In [10]:
new_recipe1 = split_ingredient(recipe)
new_recipe1

  0%|          | 0/933 [00:00<?, ?it/s]

100%|██████████| 933/933 [00:01<00:00, 473.37it/s]

Row 29: 
Row 30: 
Row 40: 
Row 207: ▶재료:방아잎50g
Row 209: 3층치즈2장
Row 321: 
Row 335: 
Row 383: 
Row 415: 饼干碎1t
Row 633: 4인분카레1봉지
Row 932: (다시마


,recipe_title,recipe_ingredients,ingredient1,quantity1,unit1,ingredient2,quantity2,unit2,ingredient3,quantity3,...,unit21,ingredient22,quantity22,unit22,ingredient23,quantity23,unit23,ingredient24,quantity24,unit24
0,호박 새우젓찌개 만드는 방법~★,"{'재료': ['둥근호박1개', '청양고추3개', '당근약간', '실파약간', '새...",둥근호박,1,개,청양고추,3,개,당근약간,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,바글바글 전복 된장찌개,"{'재료': ['된장1스푼', '전복'], '부재료와 양념': ['양파1/2개', ...",된장,1,스푼,전복,,None,양파,1/2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,"{'재료': ['순두부(연두부)', '바지락30g', '미더덕30g', '멸치육수(...",순두부(연두부),,None,바지락,30,g,미더덕,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,정통 바지락 순두부 찌개,"{'주재료': ['순두부1봉', '바지락100g', '양파50g', '애호박10g'...",순두부,1,봉,바지락,100,g,양파,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,크림토마토 비프 스튜,"{'주재료': ['쇠고기등심250g', '파프리카1개', '양파1/3개', '마늘2...",쇠고기등심,250,g,파프리카,1,개,양파,1/3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,버섯 와인 닭조림,"{'재료': ['토종닭1마리', '백일송이1봉지', '베이컨10장', '그린빈스',...",토종닭,1,마리,백일송이,1,봉지,베이컨,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,복숭아 우유 아이스크림~*,"{'재료': ['우유1종이컵', '복숭아1개', '연유취향껏']}",우유,1,종이컵,복숭아,1,개,연유취향껏,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,토마토 아이스바,"{'재료': ['토마토2개', '꿀2스푼', '소금1꼬집', '레몬즙(생략가능)1작...",토마토,2,개,꿀,2,스푼,소금,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,< 간식> 감자칩 / 전자렌지,"{'재료': ['감자3개'], '양념': ['케첩적당량']}",감자,3,개,케첩적당량,,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
units = [recipe[f'unit{i}'] for i in range(1, 13)]
units

combined_units = []
for unit_tuple in zip(*units):
    for unit in unit_tuple:
        if unit:  # 빈 값이 아닌 경우에만 추가
            combined_units.append(unit)

# 중복 제거
unique_units = list(set(combined_units))

from collections import Counter

unit_counts = Counter(combined_units)
for unit, count in unit_counts.items():
    if count >= 50:
        print(f"{unit}: {count}번")



개: 1383번
큰술: 852번
ml: 121번
스푼: 247번
모: 72번
g: 593번
T: 528번
대: 108번
t: 84번
컵: 378번
작은술: 126번
줌: 154번
마리: 98번
장: 145번
숟가락: 187번
공기: 74번


In [32]:
ingreds = [new_recipe1[f'ingredient{i}'] for i in range(1, 14)]

combined_ingred = []
for ingred_tuple in zip(*ingreds):
    for ingred in ingred_tuple:
        if ingred:  # 빈 값이 아닌 경우에만 추가
            combined_ingred.append(ingred)

# 중복 제거
unique_ingred = list(set(combined_ingred))

from collections import Counter

ingred_counts = Counter(combined_ingred)
for ingred, count in ingred_counts.items():
    if count >= 30:
        print(f"{ingred}: {count}번")

청양고추: 107번
물: 265번
된장: 47번
양파: 325번
감자: 108번
다진마늘: 236번
대파: 203번
고추가루: 48번
애호박: 53번
파: 30번
달걀: 86번
마늘: 75번
고춧가루: 141번
고추장: 114번
홍고추: 57번
참기름: 146번
설탕: 214번
소금약간: 87번
후추약간: 72번
간장: 173번
소금: 142번
두부: 85번
당근: 116번
진간장: 43번
올리고당: 65번
맛술: 87번
굴소스: 31번
양배추: 38번
식용유: 71번
표고버섯: 38번
김치: 50번
콩나물: 31번
호박: 31번
우유: 51번
통깨: 33번
청주: 35번
무: 33번
들기름: 30번
돼지고기: 33번
국간장: 50번
계란: 91번
후추: 46번
부추: 33번
부침가루: 54번
밀가루: 30번
밥: 77번
버터: 47번
마요네즈: 36번
식초: 33번
쌀: 48번


In [45]:
# 4. Matrix 변환
def recipe_food_matrix(data):
    data.index = range(len(data))
    def convert_fraction_to_float(quantity):
        from fractions import Fraction

        try:
            return float(Fraction(quantity))
        except ValueError:
            return None 
    def convert_unit_to_number(unit):
        '''
        단위에 따른 g 수 변환
        '''
        unit_conversion = {
            'g': 1,
            '개': 100,
            '조금' :10
        }
        return unit_conversion.get(unit, 1)
    ingredient_columns = data.filter(like='ingredient').drop(columns=['recipe_ingredients'])
    all_ingredients = set()
    for i in range(1, 21):  
        all_ingredients.update(data[f'ingredient{i}'].dropna().unique())

    col_name = ['recipe_title'].append(list(all_ingredients))
    recipe_ingredients_df = pd.DataFrame(columns=col_name)

    recipe_rows = []
    for idx, row in data.iterrows():
        recipe_data = {ingredient: 0.0 for ingredient in all_ingredients}  # 모든 식재료를 None으로 초기화
        for i in range(1, 21):  
            ingredient = row[f'ingredient{i}']
            quantity = row[f'quantity{i}']
            unit = row[f'unit{i}']
            if pd.notna(ingredient) and pd.notna(quantity):
                quantity_float = convert_fraction_to_float(quantity)
                if quantity_float is not None:
                    unit_number = convert_unit_to_number(unit) if pd.notna(unit) else 1
                    recipe_data[ingredient] = quantity_float * unit_number
        recipe_rows.append(recipe_data)

    # 새로운 데이터프레임 생성 (모든 식재료를 열로 가짐)
    recipe_ingredients_df = pd.concat([pd.DataFrame([row]) for row in recipe_rows], ignore_index=True)
    recipe_ingredients_df = recipe_ingredients_df.astype('float64')
    recipe_ingredients_df['recipe_title'] = data['recipe_title']

    # RECIPE_TITLE 컬럼을 젤 앞으로
    recipe_ingredients_df = recipe_ingredients_df[['recipe_title'] + [col for col in recipe_ingredients_df.columns if col != 'recipe_title']]

    return recipe_ingredients_df


In [11]:
# recipe_food_matrix 과정에서 recipe_title에 NaN값이 들어감
new_recipe = recipe_food_matrix(new_recipe1)
new_recipe

NameError: name 'recipe_food_matrix' is not defined

In [56]:
new_recipe.rename(columns={'recipe_title': 'RECIPE_TITLE'}, inplace=True)
new_recipe

,RECIPE_TITLE,딸기잼원하는대로,오일적당량,노른자,무썰어서,비지,양배추약간,검은깨,다시팩,천일염,...,파마산치즈가루선택,쌈장,설탕적당량,연두청양초,차돌박이,알배추,생강가루,닭볶음용,가리비,청경채선택
0,호박 새우젓찌개 만드는 방법~★,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,바글바글 전복 된장찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,정통 바지락 순두부 찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,크림토마토 비프 스튜,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,버섯 와인 닭조림,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
929,복숭아 우유 아이스크림~*,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,토마토 아이스바,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
931,< 간식> 감자칩 / 전자렌지,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
# oracle db의 열의 최대갯수 = 1000개
# 자를 필요가 있음
new_recipe1 = new_recipe.iloc[:, 0:999]
new_recipe1 = new_recipe1.astype('str')
new_recipe1


,RECIPE_TITLE,딸기잼원하는대로,오일적당량,노른자,무썰어서,비지,양배추약간,검은깨,다시팩,천일염,...,바게트,국거리소고기,파손가락한마디,채썬대파,케첩,쌀뜨물,닭안심,불린표고버섯,돼지고기앞다리살,물가지
0,호박 새우젓찌개 만드는 방법~★,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,바글바글 전복 된장찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,정통 바지락 순두부 찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,크림토마토 비프 스튜,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,버섯 와인 닭조림,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
929,복숭아 우유 아이스크림~*,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,토마토 아이스바,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
931,< 간식> 감자칩 / 전자렌지,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# NaN 값 확인
nan_values = new_recipe[new_recipe['recipe_title'].isna()]
nan_values

,recipe_title,딸기잼원하는대로,오일적당량,노른자,무썰어서,비지,양배추약간,검은깨,다시팩,천일염,...,파마산치즈가루선택,쌈장,설탕적당량,연두청양초,차돌박이,알배추,생강가루,닭볶음용,가리비,청경채선택


In [58]:
new_recipe2 = pd.concat([new_recipe.iloc[:,:1], new_recipe.iloc[:, 999:]], axis = 1)
new_recipe2 = new_recipe2.astype('str')
new_recipe2

,RECIPE_TITLE,스파게티소스,돼지고기후지,식초약간,다시마육수나생수,닭다리살,연두부,꽈리고추,까나리액젓,돼지갈비,...,파마산치즈가루선택,쌈장,설탕적당량,연두청양초,차돌박이,알배추,생강가루,닭볶음용,가리비,청경채선택
0,호박 새우젓찌개 만드는 방법~★,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,바글바글 전복 된장찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,정통 바지락 순두부 찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,크림토마토 비프 스튜,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,버섯 와인 닭조림,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
929,복숭아 우유 아이스크림~*,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,토마토 아이스바,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
931,< 간식> 감자칩 / 전자렌지,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 컬럼갯수가 2000개가 넘어갈 시 사용하는 코드
new_recipe3 = pd.concat([new_recipe.iloc[:,:1], new_recipe.iloc[:, :]], axis = 1)
new_recipe3 = new_recipe3.astype('str')
new_recipe3

,RECIPE_TITLE,돼지고기(부위상관없음),매실엑기스,식빵(또는좋아하는빵),생강조청,보리쌀,조랭이떡(떡볶이떡가능),바닐라아이스크림,양파작은사이즈,멸치,...,깐호두,생강술,소고기샤브샤브용,빵가루약간,알쿠니아황도,삶은메추리알,면육수,오이네개,옥수수캔,파프리카노랑
0,나 순대라고 말 못해요! HACCP인증 제품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,시래기 된장국,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,바질크림파스타만들기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,돼지 수육 삶는법? 보쌈 삶지 말고 찌세요,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,브런치 참치에그샐러드만들기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
732,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
838,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
910,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
# create table
# db connection
od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12")
conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],
                    dsn = config.DB_CONFIG['dsn'])


# 테이블 이름 및 컬럼 정보
table_name = 'recipe_ingre2'
columns_info = ', '.join([f'"{col}" VARCHAR2(300)' for col in new_recipe2.columns])

# 테이블 생성
create_table_sql = f"CREATE TABLE {table_name} ({columns_info})"

exe = conn.cursor()
exe.execute(create_table_sql)
exe.close()

In [62]:
# db connection
od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12")
conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],
                    dsn = config.DB_CONFIG['dsn'])

table_name = 'recipe_ingre2'

# oracle db에 insert 
def insert_into_oracle(dataframe, table_name, conn):    
    exe = conn.cursor()    
    
    insert = [tuple(x) for x in dataframe.values]
    exe.executemany(
        f"INSERT INTO {table_name} VALUES ({','.join([':' + str(i+1) for i in range(len(dataframe.columns))])})",
        insert)
    
    conn.commit()
    exe.close()
    
    
insert_into_oracle(new_recipe2, table_name, conn)
conn.close()

In [63]:
# JOIN으로 레시피별 식재료 테이블 1개로 출력
query = 'select A.*, B.* FROM recipe_ingre1 A LEFT JOIN recipe_ingre2 B ON A.recipe_title = B.recipe_title'
result = select_table(query)
result

,recipe_title,딸기잼원하는대로,오일적당량,노른자,무썰어서,비지,양배추약간,검은깨,다시팩,천일염,...,파마산치즈가루선택,쌈장,설탕적당량,연두청양초,차돌박이,알배추,생강가루,닭볶음용,가리비,청경채선택
0,호박 새우젓찌개 만드는 방법~★,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,바글바글 전복 된장찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,제육볶음 맛있게 만드는 법,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,냉이 된장찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,팽이버섯 돼지고기볶음,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,홈메이드 과일빙수~ 푸짐해요!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
953,< 면 요리 > 새우 크림 파스타,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
954,토마토소스 폭찹,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
955,고등어 조림,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0


In [65]:
# 중복 체크
duplicates = result[result.duplicated(['recipe_title'], keep=False)]
duplicates

,recipe_title,딸기잼원하는대로,오일적당량,노른자,무썰어서,비지,양배추약간,검은깨,다시팩,천일염,...,파마산치즈가루선택,쌈장,설탕적당량,연두청양초,차돌박이,알배추,생강가루,닭볶음용,가리비,청경채선택
3,냉이 된장찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,냉이 된장찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,냉이 된장찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,청국장찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,돼지고기 김치찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57,방울토마토 마리네이드,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,방울토마토 마리네이드,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,청국장찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,청국장찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,오꼬노미야끼,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
new_recipe1

,recipe_title,recipe_ingredients,ingredient1,quantity1,unit1,ingredient2,quantity2,unit2,ingredient3,quantity3,...,unit21,ingredient22,quantity22,unit22,ingredient23,quantity23,unit23,ingredient24,quantity24,unit24
0,호박 새우젓찌개 만드는 방법~★,"{'재료': ['둥근호박1개', '청양고추3개', '당근약간', '실파약간', '새...",둥근호박,1,개,청양고추,3,개,당근약간,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,바글바글 전복 된장찌개,"{'재료': ['된장1스푼', '전복'], '부재료와 양념': ['양파1/2개', ...",된장,1,스푼,전복,,None,양파,1/2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,"{'재료': ['순두부(연두부)', '바지락30g', '미더덕30g', '멸치육수(...",순두부(연두부),,None,바지락,30,g,미더덕,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,정통 바지락 순두부 찌개,"{'주재료': ['순두부1봉', '바지락100g', '양파50g', '애호박10g'...",순두부,1,봉,바지락,100,g,양파,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,크림토마토 비프 스튜,"{'주재료': ['쇠고기등심250g', '파프리카1개', '양파1/3개', '마늘2...",쇠고기등심,250,g,파프리카,1,개,양파,1/3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,버섯 와인 닭조림,"{'재료': ['토종닭1마리', '백일송이1봉지', '베이컨10장', '그린빈스',...",토종닭,1,마리,백일송이,1,봉지,베이컨,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,복숭아 우유 아이스크림~*,"{'재료': ['우유1종이컵', '복숭아1개', '연유취향껏']}",우유,1,종이컵,복숭아,1,개,연유취향껏,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,토마토 아이스바,"{'재료': ['토마토2개', '꿀2스푼', '소금1꼬집', '레몬즙(생략가능)1작...",토마토,2,개,꿀,2,스푼,소금,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,< 간식> 감자칩 / 전자렌지,"{'재료': ['감자3개'], '양념': ['케첩적당량']}",감자,3,개,케첩적당량,,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
import pandas as pd

# txt 파일 경로
file_path = r"C:\Users\admin\OneDrive\바탕 화면\change2.txt"

# 빈 리스트 초기화
data = []

# 텍스트 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 각 줄에 대해 처리
for line in lines:
    # 공백을 기준으로 열과 값 분리
    parts = line.split()    
    # 딕셔너리로 저장
    row_data = {'ingredients': parts[0], 'unit': parts[1], 'value': parts[2]}    
    # 리스트에 추가
    data.append(row_data)

df11  = pd.DataFrame(data)
# dict으로 저장해서 속도 향상
df11_dict = df11.set_index(['ingredients', 'unit']).to_dict()['value']

for index, row in new_recipe1.iterrows():
    for i in range(1, 25):
        ingredient_col = f"ingredient{i}"
        quantity_col = f"quantity{i}"
        unit_col = f"unit{i}"

        ingredient_value = row[ingredient_col] # ingredient{i} 행 데이터
        unit_value = row[unit_col] # unit_col{i} 행 데이터

        # df11_dict에서 일치하는 값을 찾아서 new_recipe1에 채우기
        if (ingredient_value, unit_value) in df11_dict:
            new_recipe1.at[index, unit_col] = df11_dict[(ingredient_value, unit_value)]
            

In [17]:
new_recipe1

,recipe_title,recipe_ingredients,ingredient1,quantity1,unit1,ingredient2,quantity2,unit2,ingredient3,quantity3,...,unit21,ingredient22,quantity22,unit22,ingredient23,quantity23,unit23,ingredient24,quantity24,unit24
0,호박 새우젓찌개 만드는 방법~★,"{'재료': ['둥근호박1개', '청양고추3개', '당근약간', '실파약간', '새...",둥근호박,1,None,청양고추,3,None,당근약간,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,바글바글 전복 된장찌개,"{'재료': ['된장1스푼', '전복'], '부재료와 양념': ['양파1/2개', ...",된장,1,스푼,전복,,None,양파,1/2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,"{'재료': ['순두부(연두부)', '바지락30g', '미더덕30g', '멸치육수(...",순두부(연두부),,None,바지락,30,1,미더덕,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,정통 바지락 순두부 찌개,"{'주재료': ['순두부1봉', '바지락100g', '양파50g', '애호박10g'...",순두부,1,봉,바지락,100,1,양파,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,크림토마토 비프 스튜,"{'주재료': ['쇠고기등심250g', '파프리카1개', '양파1/3개', '마늘2...",쇠고기등심,250,g,파프리카,1,100,양파,1/3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,버섯 와인 닭조림,"{'재료': ['토종닭1마리', '백일송이1봉지', '베이컨10장', '그린빈스',...",토종닭,1,마리,백일송이,1,봉지,베이컨,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,복숭아 우유 아이스크림~*,"{'재료': ['우유1종이컵', '복숭아1개', '연유취향껏']}",우유,1,종이컵,복숭아,1,2,연유취향껏,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,토마토 아이스바,"{'재료': ['토마토2개', '꿀2스푼', '소금1꼬집', '레몬즙(생략가능)1작...",토마토,2,50,꿀,2,스푼,소금,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,< 간식> 감자칩 / 전자렌지,"{'재료': ['감자3개'], '양념': ['케첩적당량']}",감자,3,150,케첩적당량,,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:

pd.set_option('display.max_rows',None) # 전체 행 출력하기

In [34]:
new_recipe1['ingredient1']

0               둥근호박
1                 된장
2           순두부(연두부)
3                순두부
4              쇠고기등심
5          돼지고기(찌개용)
6             알래스카연어
7              간돼지고기
8                 두부
9         돼지고기(뒷다리살)
11            돼지고기목살
12                낙지
13              닭가슴살
14                낙지
15              닭안심살
16                 물
17            돼지고기목살
18            돼지고기목살
19                양파
20             배추우거지
21               애호박
22              열무김치
23               오징어
24             생물주꾸미
25             소고기다짐
26                두부
27              닭가슴살
28           돼지고기앞다리
29                두부
30              돼지갈비
31               청국장
32                된장
33              닭가슴살
34                꾸미
35            훈제오리고기
36                감자
37                두부
39         소고기(불고기용)
40               삼겹살
41       청정원멸치국물내기티백
42                 햄
43               콩나물
44              조랭이떡
45              돼지갈비
46              감자큰거
47               삼겹살
48                호박
49           

In [35]:
df11['ingredients']

0         애호박
1          달걀
2          양파
3        꽈리고추
4          가지
5        청양고추
6         홍고추
7          감자
8        브로콜리
9          쪽파
10      새송이버섯
11         오이
12        통마늘
13       붉은고추
14      양송이버섯
15        청피망
16         무우
17       표고버섯
18         당근
19         대파
20         고추
21        청경채
22         계란
23     미니파프리카
24       파프리카
25      방울토마토
26      양파(중)
27         스팸
28         마늘
29         호박
30     비엔나소세지
31         어묵
32      감자(중)
33        참치캔
34         맛살
35     감자중간크기
36       마른고추
37         땡초
38        청양초
39      양파작은것
40          파
41         피망
42       감자큰것
43     노랑파프리카
44     빨강파프리카
45      양파작은거
46      양파(소)
47      감자작은것
48        소세지
49        청고추
50        홍피망
51     빨간파프리카
52     노란파프리카
53       팽이버섯
54        새송이
55      건표고버섯
56       청량고추
57     비엔나소시지
58        토마토
59          무
60         유부
61       매운고추
62        건고추
63        풋고추
64       다진마늘
65      페퍼론치노
66     다진청양고추
67        고구마
68         비트
69          햄
70        크래미
71    

In [13]:
# 원래 사용하던 방식(실행x)
# txt 파일 경로 (딕셔너리 수정시 수정 필요함)
file_path = r"models/data/change.txt"
unit_conversion = {}
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip().split()
        unit = line[0]
        value = line[1] if line[1].isdigit() else None
        unit_conversion[unit] = value

# DataFrame 생성
df11 = pd.DataFrame(list(unit_conversion.items()), columns=['unit', 'value'])

# DataFrame을 딕셔너리로 변환
df11_dict = df11.set_index('unit')['value'].to_dict()

# 딕셔너리의 값을 숫자로 변환하여 새로운 딕셔너리 생성
df_dict = {key: int(value) if value is not None else None for key, value in df11_dict.items()}
df_dict

# unit{i} 컬럼에 딕셔너리로 지정한 key : value값으로 치환
for i in range(1, 15):
    column_name = f'unit{i}'
    if column_name in new_recipe1.columns:
        new_recipe1[column_name] = new_recipe1[column_name].apply(lambda x: df_dict.get(x) if pd.notna(x) and not re.match(r'\d+[^\d]*$', str(x)) else x)

{'g': '1', 'ml': '1', '작은술': '5', '큰술': '15', '팩': '100', 'T': '15', '주먹': '50', '조각': '40', 'L': '10', '알': '60', '줄': '100', '인분': None, '개': '100', '덩이': '200', '줌': '50', '스푼': '15', '장': None, '공기': '210', '마리': '800', '종이컵': '180', '수저': '15', '꼬집': '2', '모': '300', '쪽': '5', '컵': '240', '캔': '350', '포기': '3000', '움쿰': '50', 't': '5', '덩어리': '100', '개약': '1', 'cc': '1', '개씩': '1', '대': '30', '숟가락': '15', '줄기': '10', '번톡톡': '5', '통': '100', 'ts': '5', '묶음': '200', 'kg': None, '뿌리': '5', 'TS': '5', '리터': None, '그람': '1', '봉지': '100', 'cm': '3', '개분량': '100', '작은숟가락': '5', '취향껏': '50', '그램': '1', 'T스푼': '15', '움큼': '50', '스틱': '5', '슬라이스': '10', 'g정도': '1', '바퀴': '10', '토막': '40', '웅큼': '50', '단': '400', 'ea': '100', '종': '50', '키로': '1000', '약간': '10', '겁': '240', '병': '1000', '냄비': None, 'or단호박': None, 'C': '240', '티스푼': '5', 'Broth': '1000', '닭육수농축액이있을경우': None, '만약농축액이없고육수로만할때육수': None, 'WhippingCream일인용기준': None, 'or꿀약간': None, 'gg': '1', '개정도': '50', '컵이상': '240', '소주잔컵': '50'

In [153]:
new_recipe1 = new_recipe1[['recipe_title'] + [f'{name}{i}' for i in range(1, 14) for name in ['ingredient', 'quantity', 'unit']]]
new_recipe1

,recipe_title,ingredient1,quantity1,unit1,ingredient2,quantity2,unit2,ingredient3,quantity3,unit3,...,unit10,ingredient11,quantity11,unit11,ingredient12,quantity12,unit12,ingredient13,quantity13,unit13
0,호박 새우젓찌개 만드는 방법~★,둥근호박,1,100.0,청양고추,3,100.0,당근약간,,NaN,...,NaN,None,None,NaN,None,None,NaN,None,None,NaN
1,바글바글 전복 된장찌개,된장,1,15.0,전복,,NaN,양파,1/2,100.0,...,NaN,고추가루,,NaN,None,None,NaN,None,None,NaN
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,순두부(연두부),,NaN,바지락,30,1.0,미더덕,30,1.0,...,15.0,생강즙,1,5.0,고춧가루,1,15.0,고추장,1,15.0
3,정통 바지락 순두부 찌개,순두부,1,100.0,바지락,100,1.0,양파,50,1.0,...,100.0,다시다명품골드쇠고기,15,1.0,고운고춧가루,5,1.0,참기름,5,1.0
4,크림토마토 비프 스튜,쇠고기등심,250,1.0,파프리카,1,100.0,양파,1/3,100.0,...,NaN,후추적당량,,NaN,이탈리안파슬리약간,,NaN,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,버섯 와인 닭조림,토종닭,1,800.0,백일송이,1,100.0,베이컨,10,NaN,...,1000.0,마늘,,NaN,소금,,NaN,후추,,NaN
996,복숭아 우유 아이스크림~*,우유,1,180.0,복숭아,1,100.0,연유취향껏,,NaN,...,NaN,None,None,NaN,None,None,NaN,None,None,NaN
997,토마토 아이스바,토마토,2,100.0,꿀,2,15.0,소금,1,2.0,...,NaN,None,None,NaN,None,None,NaN,None,None,NaN
998,< 간식> 감자칩 / 전자렌지,감자,3,100.0,케첩적당량,,NaN,None,None,NaN,...,NaN,None,None,NaN,None,None,NaN,None,None,NaN


In [82]:
new_recipe1['unit1']

0      100.0
1       15.0
2        NaN
3      100.0
4        1.0
       ...  
995    800.0
996    180.0
997    100.0
998    100.0
999    100.0
Name: unit1, Length: 933, dtype: float64

In [154]:
# 타입변경 str => float
for i in range(1, 14):
    try:
        new_recipe1[f'quantity{i}'] = pd.to_numeric(new_recipe1[f'quantity{i}'], errors='coerce').astype('float16')
    except ValueError:
        new_recipe1[f'quantity{i}'] = 0


C:\Users\admin\AppData\Local\Temp\ipykernel_24420\1244780534.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_recipe1[f'quantity{i}'] = pd.to_numeric(new_recipe1[f'quantity{i}'], errors='coerce').astype('float16')
C:\Users\admin\AppData\Local\Temp\ipykernel_24420\1244780534.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_recipe1[f'quantity{i}'] = pd.to_numeric(new_recipe1[f'quantity{i}'], errors='coerce').astype('float16')
C:\Users\admin\AppData\Local\Temp\ipykernel_24420\1244780534.py:4: Set

In [155]:
# mulit{i} 컬럼 생성 후 quantity * unit 값 대입
for i in range(1,14):
    new_recipe1[f'multi{i}'] = None
    
for i in range(1, 14):
    new_recipe1[f'multi{i}'] = new_recipe1[f'quantity{i}'] * new_recipe1[f'unit{i}']

C:\Users\admin\AppData\Local\Temp\ipykernel_24420\328669919.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_recipe1[f'multi{i}'] = None
C:\Users\admin\AppData\Local\Temp\ipykernel_24420\328669919.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_recipe1[f'multi{i}'] = None
C:\Users\admin\AppData\Local\Temp\ipykernel_24420\328669919.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [156]:
# quantity, unit 컬럼 전부 삭제
for i in range(1,14):
    new_recipe1 = new_recipe1.drop(f'quantity{i}',axis = 1)
    new_recipe1 = new_recipe1.drop(f'unit{i}',axis = 1)

In [158]:
new_columns = [
    'recipe_title', 
    'ingredient1', 'multi1', 
    'ingredient2', 'multi2', 
    'ingredient3', 'multi3', 
    'ingredient4', 'multi4', 
    'ingredient5', 'multi5', 
    'ingredient6', 'multi6', 
    'ingredient7', 'multi7', 
    'ingredient8', 'multi8', 
    'ingredient9', 'multi9', 
    'ingredient10', 'multi10', 
    'ingredient11', 'multi11', 
    'ingredient12', 'multi12', 
    'ingredient13', 'multi13'
]

new_recipe1 = new_recipe1[new_columns]

new_recipe1.columns

Index(['recipe_title', 'ingredient1', 'multi1', 'ingredient2', 'multi2',
       'ingredient3', 'multi3', 'ingredient4', 'multi4', 'ingredient5',
       'multi5', 'ingredient6', 'multi6', 'ingredient7', 'multi7',
       'ingredient8', 'multi8', 'ingredient9', 'multi9', 'ingredient10',
       'multi10', 'ingredient11', 'multi11', 'ingredient12', 'multi12',
       'ingredient13', 'multi13'],
      dtype='object')

In [159]:
# 컬럼명 변경
nutri_df.rename(columns={'대표식품명':'ingredient'}, inplace=True)

In [57]:
new_recipe1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 933 entries, 0 to 999
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   recipe_title  933 non-null    object 
 1   ingredient1   932 non-null    object 
 2   multi1        665 non-null    float64
 3   ingredient2   922 non-null    object 
 4   multi2        529 non-null    float64
 5   ingredient3   902 non-null    object 
 6   multi3        510 non-null    float64
 7   ingredient4   860 non-null    object 
 8   multi4        449 non-null    float64
 9   ingredient5   801 non-null    object 
 10  multi5        450 non-null    float64
 11  ingredient6   731 non-null    object 
 12  multi6        407 non-null    float64
 13  ingredient7   642 non-null    object 
 14  multi7        354 non-null    float64
 15  ingredient8   557 non-null    object 
 16  multi8        325 non-null    float64
 17  ingredient9   475 non-null    object 
 18  multi9        271 non-null    float

In [89]:
# 한번에 처리 시도
# 첫번째 테이블
new_recipe1

# 두번째 테이블
nutri_df


for i in range(1, 14):  # ingredient1부터 ingredient13까지 처리
    ingredient_col = f'ingredient{i}'
    multi_col = f'multi{i}'
    
    # 필요한 컬럼만 추출하여 병합
    merged_df = pd.merge(new_recipe1[['recipe_title', ingredient_col, multi_col]],
                         nutri_df,
                         left_on=ingredient_col,
                         right_on='ingredient',
                         how='left')  # inner에서 left로 변경
    
    # 각 값에 대해 계산
    for index, row in merged_df.iterrows():
        if pd.notna(row['ingredient']):  # nutrient_table에 일치하는 값이 있을 때만 계산 수행
            multiplier = row[multi_col] / 100 # row[i]
            for nutrient in ['에너지(kcal)', '수분(g)', '단백질(g)',
               '지방(g)', '회분(g)', '탄수화물(g)', '당류(g)', '식이섬유(g)', '칼슘(mg)', '철(mg)',
               '인(mg)', '칼륨(mg)', '나트륨(mg)', '비타민a(μg rae)', '레티놀(μg)', '베타카로틴(μg)',
               '티아민(mg)', '리보플라빈(mg)', '니아신(mg)', '비타민c(mg)', '비타민d(μg)', '콜레스테롤(mg)',
               '포화지방산(g)', '트랜스지방산(g)', '폐기율(%)']:
                new_recipe1.at[index, f'{nutrient}{i}'] = row[nutrient] * multiplier
        else:
            for nutrient in ['에너지(kcal)', '수분(g)', '단백질(g)',
               '지방(g)', '회분(g)', '탄수화물(g)', '당류(g)', '식이섬유(g)', '칼슘(mg)', '철(mg)',
               '인(mg)', '칼륨(mg)', '나트륨(mg)', '비타민a(μg rae)', '레티놀(μg)', '베타카로틴(μg)',
               '티아민(mg)', '리보플라빈(mg)', '니아신(mg)', '비타민c(mg)', '비타민d(μg)', '콜레스테롤(mg)',
               '포화지방산(g)', '트랜스지방산(g)', '폐기율(%)']:
                new_recipe1.at[index, f'{nutrient}{i}'] = None  # 또는 0 또는 다른 값으로 설정할 수 있음

# 결과 출력
new_recipe1

C:\Users\admin\AppData\Local\Temp\ipykernel_24420\1370211004.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_recipe1.at[index, f'{nutrient}{i}'] = None  # 또는 0 또는 다른 값으로 설정할 수 있음
C:\Users\admin\AppData\Local\Temp\ipykernel_24420\1370211004.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_recipe1.at[index, f'{nutrient}{i}'] = None  # 또는 0 또는 다른 값으로 설정할 수 있음
C:\Users\admin\AppData\Local\Temp\ipykernel_24420\1370211004.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling

,recipe_title,ingredient1,multi1,ingredient2,multi2,ingredient3,multi3,ingredient4,multi4,ingredient5,...,베타카로틴(μg)13,티아민(mg)13,리보플라빈(mg)13,니아신(mg)13,비타민c(mg)13,비타민d(μg)13,콜레스테롤(mg)13,포화지방산(g)13,트랜스지방산(g)13,폐기율(%)13
0,호박 새우젓찌개 만드는 방법~★,둥근호박,100.0,청양고추,300.0,당근약간,NaN,실파약간,NaN,새우젓,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,바글바글 전복 된장찌개,된장,15.0,전복,NaN,양파,NaN,둥근호박,NaN,감자,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,순두부(연두부),NaN,바지락,30.0,미더덕,30.0,멸치육수(만능육수),NaN,고추기름,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,정통 바지락 순두부 찌개,순두부,100.0,바지락,100.0,양파,50.0,애호박,10.0,굵은고춧가루,...,0.55,0.0,0.0035,0.0,0.0,0.0,0.0,0.7325,0.0145,0.0
4,크림토마토 비프 스튜,쇠고기등심,250.0,파프리카,100.0,양파,NaN,마늘,10.0,새송이버섯,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
# nan값 제거
new_recipe1 = new_recipe1.dropna(subset=['recipe_title'])

In [91]:
new_recipe1

,recipe_title,ingredient1,multi1,ingredient2,multi2,ingredient3,multi3,ingredient4,multi4,ingredient5,...,베타카로틴(μg)13,티아민(mg)13,리보플라빈(mg)13,니아신(mg)13,비타민c(mg)13,비타민d(μg)13,콜레스테롤(mg)13,포화지방산(g)13,트랜스지방산(g)13,폐기율(%)13
0,호박 새우젓찌개 만드는 방법~★,둥근호박,100.0,청양고추,300.0,당근약간,NaN,실파약간,NaN,새우젓,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,바글바글 전복 된장찌개,된장,15.0,전복,NaN,양파,NaN,둥근호박,NaN,감자,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,순두부(연두부),NaN,바지락,30.0,미더덕,30.0,멸치육수(만능육수),NaN,고추기름,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,정통 바지락 순두부 찌개,순두부,100.0,바지락,100.0,양파,50.0,애호박,10.0,굵은고춧가루,...,0.55,0.0,0.0035,0.0,0.0,0.0,0.0,0.7325,0.0145,0.0
4,크림토마토 비프 스튜,쇠고기등심,250.0,파프리카,100.0,양파,NaN,마늘,10.0,새송이버섯,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,버섯 와인 닭조림,토종닭,800.0,백일송이,100.0,베이컨,NaN,그린빈스,NaN,브로콜리,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,복숭아 우유 아이스크림~*,우유,180.0,복숭아,100.0,연유취향껏,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,토마토 아이스바,토마토,200.0,꿀,30.0,소금,2.0,레몬즙(생략가능),5.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,< 간식> 감자칩 / 전자렌지,감자,300.0,케첩적당량,NaN,None,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# 중복 체크
duplicates = new_recipe1[new_recipe1.duplicated(['recipe_title'], keep=False)]
all_duplicates = new_recipe1[new_recipe1['recipe_title'].isin(duplicates['recipe_title'])]

In [62]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(all_duplicates)

    recipe_title    ingredient1  multi1 ingredient2  multi2 ingredient3  \
21       냉이 된장찌개            애호박     NaN          두부     NaN          냉이   
31         청국장찌개            청국장   100.0       느타리버섯    50.0      고기간거조금   
83     돼지고기 김치찌개           돼지고기   250.0          김치     NaN           파   
110       오꼬노미야끼  백설쿠킷오꼬노미야끼KIT   100.0         양배추   200.0      햄스빌베이컨   
170  방울토마토 마리네이드          방울토마토  3000.0          양파    15.0        올리브유   
171  방울토마토 마리네이드          방울토마토  3000.0          양파    15.0        올리브유   
294       오꼬노미야끼             치즈     NaN         베이컨     NaN         양배추   
343      냉이 된장찌개             냉이   100.0          두부     NaN          양파   
448        청국장찌개            신김치    50.0     돼지고기찌개용    50.0          두부   
463        청국장찌개             두부    50.0          김치     NaN         쌀뜨물   
500    돼지고기 김치찌개             김치     NaN        돼지고기     NaN          두부   
600      냉이 된장찌개             냉이    50.0         소고기    50.0           물   
704  간편하게 끓인 문어죽         

In [22]:
# 중복 제거(실행은 안함)
new_recipe1 = new_recipe1.drop_duplicates(subset=['recipe_title'])


In [92]:
# ingredient, multi 컬럼 전부 삭제
for i in range(1,14):
    new_recipe1 = new_recipe1.drop(f'ingredient{i}',axis = 1)
    new_recipe1 = new_recipe1.drop(f'multi{i}',axis = 1)

In [93]:
new_recipe1

,recipe_title,에너지(kcal)1,수분(g)1,단백질(g)1,지방(g)1,회분(g)1,탄수화물(g)1,당류(g)1,식이섬유(g)1,칼슘(mg)1,...,베타카로틴(μg)13,티아민(mg)13,리보플라빈(mg)13,니아신(mg)13,비타민c(mg)13,비타민d(μg)13,콜레스테롤(mg)13,포화지방산(g)13,트랜스지방산(g)13,폐기율(%)13
0,호박 새우젓찌개 만드는 방법~★,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,바글바글 전복 된장찌개,21.3,8.73,2.04,1.0335,2.1105,1.089,0.0615,0.72,17.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,정통 바지락 순두부 찌개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.55,0.0,0.0035,0.0,0.0,0.0,0.0,0.7325,0.0145,0.0
4,크림토마토 비프 스튜,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,버섯 와인 닭조림,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,복숭아 우유 아이스크림~*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,토마토 아이스바,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,< 간식> 감자칩 / 전자렌지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# recipe_title 컬럼명 대문자로 변경
new_recipe1 = new_recipe1.rename(columns={'recipe_title': 'RECIPE_TITLE'})

In [67]:
# db connection
od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12")
conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],
                    dsn = config.DB_CONFIG['dsn'])


# 테이블 이름 및 컬럼 정보 추출
table_name = 'recipe_nutri_all'
columns_info = ', '.join([f'"{col}" VARCHAR2(300)' for col in new_recipe1.columns])

# 테이블 생성
create_table_sql = f"CREATE TABLE {table_name} ({columns_info})"

exe = conn.cursor()
exe.execute(create_table_sql)
exe.close()

In [70]:
# insert 하기 위해 데이터 타입 변경(밑에 insert 할때만 실행. 안할시 실행x)
new_recipe1 = new_recipe1.astype('str')

In [71]:
# 테이블 이름 및 컬럼 정보 추출
table_name = 'recipe_nutri_all'

# insert 
def insert_into_oracle(dataframe, table_name, conn):    
    exe = conn.cursor()    

    # 컬럼 수에 따라 바인드 변수 및 데이터 생성
    bind_vars = ','.join([':' + str(i + 1) for i in range(len(dataframe.columns))])
    data_to_insert = [tuple(row) for _, row in dataframe.iterrows()]

    # executemany를 사용하여 여러 행을 한 번에 삽입
    exe.executemany(f"INSERT INTO {table_name} VALUES ({bind_vars})", data_to_insert)
    
    conn.commit()
    exe.close()    
    
# insert 실행
insert_into_oracle(new_recipe1, table_name, conn)
conn.close()

In [95]:
# recipe_title 컬럼명 소문자로 변경
new_recipe1 = new_recipe1.rename(columns={'RECIPE_TITLE': 'recipe_title'})

In [96]:
# 총합 영양소 컬럼 생성
total_columns = ['총합_에너지(kcal)', '총합_수분(g)', '총합_단백질(g)', '총합_지방(g)', '총합_회분(g)', '총합_탄수화물(g)',
                '총합_당류(g)', '총합_식이섬유(g)', '총합_칼슘(mg)', '총합_철(mg)', '총합_인(mg)', '총합_칼륨(mg)',
                '총합_나트륨(mg)', '총합_비타민a(μg rae)', '총합_레티놀(μg)', '총합_베타카로틴(μg)', '총합_티아민(mg)',
                '총합_리보플라빈(mg)', '총합_니아신(mg)', '총합_비타민c(mg)', '총합_비타민d(μg)', '총합_콜레스테롤(mg)',
                '총합_포화지방산(g)', '총합_트랜스지방산(g)', '총합_폐기율(%)']

new_recipe1[total_columns] = 0

In [97]:
# 각 컬럼당 sum값을 방금 만든 총합 ~ 컬럼에 각각 적용
nutrient_columns = ['에너지(kcal)', '수분(g)', '단백질(g)', '지방(g)', '회분(g)', '탄수화물(g)', '당류(g)',
                    '식이섬유(g)', '칼슘(mg)', '철(mg)', '인(mg)', '칼륨(mg)', '나트륨(mg)', '비타민a(μg rae)',
                    '레티놀(μg)', '베타카로틴(μg)', '티아민(mg)', '리보플라빈(mg)', '니아신(mg)', '비타민c(mg)',
                    '비타민d(μg)', '콜레스테롤(mg)', '포화지방산(g)', '트랜스지방산(g)', '폐기율(%)']

for nutrient in nutrient_columns:
    new_recipe1[f'총합_{nutrient}'] = new_recipe1[[f'{nutrient}{i}' for i in range(1, 14)]].sum(axis=1)

In [98]:
columns_to_keep = ['recipe_title'] + [f'총합_{nutrient}' for nutrient in nutrient_columns]
# 남길 컬럼만 선택
new_recipe1 = new_recipe1.loc[:, columns_to_keep]

In [99]:
# 소수점 3자리까지만
new_recipe1 = round(new_recipe1, 3)

In [100]:
new_recipe1

,recipe_title,총합_에너지(kcal),총합_수분(g),총합_단백질(g),총합_지방(g),총합_회분(g),총합_탄수화물(g),총합_당류(g),총합_식이섬유(g),총합_칼슘(mg),...,총합_베타카로틴(μg),총합_티아민(mg),총합_리보플라빈(mg),총합_니아신(mg),총합_비타민c(mg),총합_비타민d(μg),총합_콜레스테롤(mg),총합_포화지방산(g),총합_트랜스지방산(g),총합_폐기율(%)
0,호박 새우젓찌개 만드는 방법~★,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.00,0.000,0.000,0.000,0.00,0.0,0.0,0.000,0.000,0.0
1,바글바글 전복 된장찌개,100.30,87.530,4.380,1.124,3.010,18.959,0.061,2.620,22.10,...,9.05,0.186,0.080,0.212,0.00,0.0,0.0,0.172,0.003,0.0
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,446.10,103.065,22.097,22.645,3.086,39.098,3.723,6.585,142.40,...,1187.55,0.442,1.118,1.864,1.32,0.0,0.0,0.250,0.008,8.4
3,정통 바지락 순두부 찌개,563.85,149.910,30.270,29.880,3.173,41.737,1.050,0.000,127.15,...,0.55,0.405,0.768,0.640,1.90,0.0,0.0,0.742,0.014,6.0
4,크림토마토 비프 스튜,529.90,13.580,6.200,38.527,2.138,49.555,0.211,0.300,160.90,...,0.00,0.061,0.104,0.538,0.16,0.0,0.0,11.315,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,버섯 와인 닭조림,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.00,0.000,0.000,0.000,0.00,0.0,0.0,0.000,0.000,0.0
996,복숭아 우유 아이스크림~*,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.00,0.000,0.000,0.000,0.00,0.0,0.0,0.000,0.000,0.0
997,토마토 아이스바,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.00,0.000,0.000,0.000,0.00,0.0,0.0,0.000,0.000,0.0
998,< 간식> 감자칩 / 전자렌지,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.00,0.000,0.000,0.000,0.00,0.0,0.0,0.000,0.000,0.0


In [102]:
# 결측치가 많아서 숫자형으로 insert 안됨. 일단 str로 변경 후 insert하는 형식
new_recipe1 = new_recipe1.astype('str')

In [104]:
# db connection
od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12")
conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],
                    dsn = config.DB_CONFIG['dsn'])

# 테이블 이름 및 컬럼 정보 추출
table_name = 'recipe_nutrient'

# insert 
def insert_into_oracle(dataframe, table_name, conn):    
    exe = conn.cursor()    

    # 컬럼 수에 따라 바인드 변수 및 데이터 생성
    bind_vars = ','.join([':' + str(i + 1) for i in range(len(dataframe.columns))])
    data_to_insert = [tuple(row) for _, row in dataframe.iterrows()]

    # executemany를 사용하여 여러 행을 한 번에 삽입
    exe.executemany(f"INSERT INTO {table_name} VALUES ({bind_vars})", data_to_insert)
    
    conn.commit()
    exe.close()    
    
# insert 실행
insert_into_oracle(new_recipe1, table_name, conn)
conn.close()

In [11]:
def recipe_nutri(new_recipe1, nutri_df):
    # txt 파일 경로 (딕셔너리 수정시 수정 필요함)
    file_path = r"C:\Users\admin\OneDrive\바탕 화면\change.txt"

    unit_conversion = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip().split()
            unit = line[0]
            value = line[1] if line[1].isdigit() else None
            unit_conversion[unit] = value

    # DataFrame 생성
    df11 = pd.DataFrame(list(unit_conversion.items()), columns=['unit', 'value'])

    # DataFrame을 딕셔너리로 변환
    df11_dict = df11.set_index('unit')['value'].to_dict()

    # 딕셔너리의 값을 숫자로 변환하여 새로운 딕셔너리 생성
    df_dict = {key: int(value) if value is not None else None for key, value in df11_dict.items()}
    df_dict

    # unit{i} 컬럼에 딕셔너리로 지정한 key : value값으로 치환
    for i in range(1, 15):
        column_name = f'unit{i}'
        if column_name in new_recipe1.columns:
            new_recipe1[column_name] = new_recipe1[column_name].apply(lambda x: df_dict.get(x) if pd.notna(x) and not re.match(r'\d+[^\d]*$', str(x)) else x)
    
    # new_recipe1에 recipe_title, ingredient{i}, quantity{i}, unit{i}만 저장
    new_recipe1 = new_recipe1[['recipe_title'] + [f'{name}{i}' for i in range(1, 14) for name in ['ingredient', 'quantity', 'unit']]]
 
    # 계산을 위해 quantity의 타입변경 str => float
    for i in range(1, 14):
        try:
            new_recipe1[f'quantity{i}'] = pd.to_numeric(new_recipe1[f'quantity{i}'], errors='coerce').astype('float16')
        except ValueError:
            new_recipe1[f'quantity{i}'] = 0
    
    # mulit{i} 컬럼 생성 후 quantity * unit 값 대입
    for i in range(1,14):
        new_recipe1[f'multi{i}'] = None
        
    for i in range(1, 14):
        new_recipe1[f'multi{i}'] = new_recipe1[f'quantity{i}'] * new_recipe1[f'unit{i}']
        
    # quantity, unit 컬럼 전부 삭제
    for i in range(1,14):
        new_recipe1 = new_recipe1.drop(f'quantity{i}',axis = 1)
        new_recipe1 = new_recipe1.drop(f'unit{i}',axis = 1)
    
    # new_recipe1의 컬럼 재배열 (recipe_title ingredient1 multi1 ... 식으로)
    new_columns = [
        'recipe_title', 
        'ingredient1', 'multi1', 
        'ingredient2', 'multi2', 
        'ingredient3', 'multi3', 
        'ingredient4', 'multi4', 
        'ingredient5', 'multi5', 
        'ingredient6', 'multi6', 
        'ingredient7', 'multi7', 
        'ingredient8', 'multi8', 
        'ingredient9', 'multi9', 
        'ingredient10', 'multi10', 
        'ingredient11', 'multi11', 
        'ingredient12', 'multi12', 
        'ingredient13', 'multi13'
    ]
    new_recipe1 = new_recipe1[new_columns]
    
    # 영양소 테이블의 컬럼명 변경
    nutri_df.rename(columns={'대표식품명':'ingredient'}, inplace=True)
    
    nutrient_list = ['에너지(kcal)', '수분(g)', '단백질(g)',
                '지방(g)', '회분(g)', '탄수화물(g)', '당류(g)', '식이섬유(g)', '칼슘(mg)', '철(mg)',
                '인(mg)', '칼륨(mg)', '나트륨(mg)', '비타민a(μg rae)', '레티놀(μg)', '베타카로틴(μg)',
                '티아민(mg)', '리보플라빈(mg)', '니아신(mg)', '비타민c(mg)', '비타민d(μg)', '콜레스테롤(mg)',
                '포화지방산(g)', '트랜스지방산(g)', '폐기율(%)']
    
    # 기존의 new_recipe1테이블(레시피명, 식재료명, multi)를 왼쪽, 영양소 테이블 nutri_df의 ingredient를 오른쪽에 두고 merge
    # for문 안에 merge가 있음. 즉 recipe_title ingredient1 multi1 ingredient 에너지(kcal) 수분(g)...
    # recipe_title ingredient2 multi2 ingredient 에너지(kcal)... 형식을 반복
    # 여기서 ingredient 열은 ingredient{i}와 nutri_df의 ingredient을 비교해서 데이터가 있으면 해당 열에 집어 넣는 형식
    # 예를들면 ingredient13에 참기름이 있는데 nutri_df에 참기름이 있으면 ingredient열에 참기름이 들어가고 
    # 없다면 NaN이 들어감
    
    # for index, row in merged_df.iterrows(): = merged_df를 행 단위로 반복
    # if pd.notna(row['ingredient']) = ingredient열 데이터가 NaN이 아니면 실행
    # multiplier = multi{i}에서 100을 나눈값 (영양소 테이블이 100g 기준이기 때문)
    # 위에 만든 nutrient_list로 for문 실행
    # new_recipe1.at[index, f'{nutrient}{i}'] = row[nutrient] * multiplier
    # = merge된 테이블에서 영양소 부분을 가져와 multiplier값을 곱하고 nutrient{i}에 저장
    # 예를들면 ingredient13이 고추장이고 영양소에서 고추장 에너지(kcal)가 100이면 100*multiplier한 값을
    # 에너지(kcal)13 에 저장하는 방식 
    
    
    for i in range(1, 14):  # ingredient1부터 ingredient13까지 처리
        ingredient_col = f'ingredient{i}'
        multi_col = f'multi{i}'
        
        # 필요한 컬럼만 추출하여 병합
        merged_df = pd.merge(new_recipe1[['recipe_title', ingredient_col, multi_col]],
                            nutri_df,
                            left_on=ingredient_col,
                            right_on='ingredient',
                            how='left')
        
        # 각 값에 대해 계산
        for index, row in merged_df.iterrows():
            if pd.notna(row['ingredient']): 
                multiplier = row[multi_col] / 100 # row[index]로 변경가능
                for nutrient in nutrient_list:
                    new_recipe1.at[index, f'{nutrient}{i}'] = row[nutrient] * multiplier
            else:
                for nutrient in nutrient_list:
                    new_recipe1.at[index, f'{nutrient}{i}'] = None  # 또는 0 또는 다른 값으로 설정할 수 있음

    # NaN값 제거
    new_recipe1 = new_recipe1.dropna(subset=['recipe_title'])
    
    # ingredient, multi 컬럼 전부 삭제
    for i in range(1,14):
        new_recipe1 = new_recipe1.drop(f'ingredient{i}',axis = 1)
        new_recipe1 = new_recipe1.drop(f'multi{i}',axis = 1)
        
    # 총합 영양소 컬럼 생성
    total_columns = ['총합_에너지(kcal)', '총합_수분(g)', '총합_단백질(g)', '총합_지방(g)', '총합_회분(g)', '총합_탄수화물(g)',
                    '총합_당류(g)', '총합_식이섬유(g)', '총합_칼슘(mg)', '총합_철(mg)', '총합_인(mg)', '총합_칼륨(mg)',
                    '총합_나트륨(mg)', '총합_비타민a(μg rae)', '총합_레티놀(μg)', '총합_베타카로틴(μg)', '총합_티아민(mg)',
                    '총합_리보플라빈(mg)', '총합_니아신(mg)', '총합_비타민c(mg)', '총합_비타민d(μg)', '총합_콜레스테롤(mg)',
                    '총합_포화지방산(g)', '총합_트랜스지방산(g)', '총합_폐기율(%)']

    new_recipe1[total_columns] = 0
    
    # 각 컬럼당 sum값을 방금 만든 총합 ~ 컬럼에 각각 적용
    nutrient_columns = ['에너지(kcal)', '수분(g)', '단백질(g)', '지방(g)', '회분(g)', '탄수화물(g)', '당류(g)',
                        '식이섬유(g)', '칼슘(mg)', '철(mg)', '인(mg)', '칼륨(mg)', '나트륨(mg)', '비타민a(μg rae)',
                        '레티놀(μg)', '베타카로틴(μg)', '티아민(mg)', '리보플라빈(mg)', '니아신(mg)', '비타민c(mg)',
                        '비타민d(μg)', '콜레스테롤(mg)', '포화지방산(g)', '트랜스지방산(g)', '폐기율(%)']

    for nutrient in nutrient_columns:
        new_recipe1[f'총합_{nutrient}'] = new_recipe1[[f'{nutrient}{i}' for i in range(1, 14)]].sum(axis=1)
    
    # 남길 컬럼만 선택
    columns_to_keep = ['recipe_title'] + [f'총합_{nutrient}' for nutrient in nutrient_columns]    
    new_recipe1 = new_recipe1.loc[:, columns_to_keep]
    
    # 소수점 3자리까지만 표시
    new_recipe1 = round(new_recipe1, 3)
    
    return new_recipe1

In [12]:
recipe_nutri(new_recipe1, nutri_df)

C:\Users\admin\AppData\Local\Temp\ipykernel_5088\3731694846.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_recipe1[f'quantity{i}'] = pd.to_numeric(new_recipe1[f'quantity{i}'], errors='coerce').astype('float16')
C:\Users\admin\AppData\Local\Temp\ipykernel_5088\3731694846.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_recipe1[f'quantity{i}'] = pd.to_numeric(new_recipe1[f'quantity{i}'], errors='coerce').astype('float16')
C:\Users\admin\AppData\Local\Temp\ipykernel_5088\3731694846.py:35: Set

,recipe_title,총합_에너지(kcal),총합_수분(g),총합_단백질(g),총합_지방(g),총합_회분(g),총합_탄수화물(g),총합_당류(g),총합_식이섬유(g),총합_칼슘(mg),...,총합_베타카로틴(μg),총합_티아민(mg),총합_리보플라빈(mg),총합_니아신(mg),총합_비타민c(mg),총합_비타민d(μg),총합_콜레스테롤(mg),총합_포화지방산(g),총합_트랜스지방산(g),총합_폐기율(%)
0,호박 새우젓찌개 만드는 방법~★,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.00,0.000,0.000,0.000,0.00,0.0,0.0,0.000,0.000,0.0
1,바글바글 전복 된장찌개,100.30,87.530,4.380,1.124,3.010,18.959,0.061,2.620,22.10,...,9.05,0.186,0.080,0.212,0.00,0.0,0.0,0.172,0.003,0.0
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,446.10,103.065,22.097,22.645,3.086,39.098,3.723,6.585,142.40,...,1187.55,0.442,1.118,1.864,1.32,0.0,0.0,0.250,0.008,8.4
3,정통 바지락 순두부 찌개,563.85,149.910,30.270,29.880,3.173,41.737,1.050,0.000,127.15,...,0.55,0.405,0.768,0.640,1.90,0.0,0.0,0.742,0.014,6.0
4,크림토마토 비프 스튜,529.90,13.580,6.200,38.527,2.138,49.555,0.211,0.300,160.90,...,0.00,0.061,0.104,0.538,0.16,0.0,0.0,11.315,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"[초기이유식 2단계] 과일퓨레 l 배퓨레, 바나나퓨레, 사과퓨레, 고구마퓨레, 단호박퓨레",0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.00,0.000,0.000,0.000,0.00,0.0,0.0,0.000,0.000,0.0
9996,가리비 버터구이,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.00,0.000,0.000,0.000,0.00,0.0,0.0,0.000,0.000,0.0
9997,시원하게 맛보는 냉우동 샐러드 만들기,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.00,0.000,0.000,0.000,0.00,0.0,0.0,0.000,0.000,0.0
9998,사위도 안 준다는 초벌 부추와 오리고기훈제,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.00,0.000,0.000,0.000,0.00,0.0,0.0,0.000,0.000,0.0
